In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [5]:
input=torch.Tensor(1,1,28,28) # 배치크기(sample 수), 채널, 높이, 너비
print(input)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
conv1=nn.Conv2d(1,32,3, padding=1) # channel, output_channel: 필터의 수, kerne_size(세로3, 가로3)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [7]:
conv2=nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [8]:
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [9]:
out=conv1(input)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [10]:
out=pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [11]:
out=conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [12]:
out=pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [13]:
out=out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [14]:
fc=nn.Linear(3136, 10)
out=fc(out)
print(out.shape)

torch.Size([1, 10])


In [15]:
print(out)
print(torch.argmax(out))

tensor([[ 0.0595,  0.1157,  0.0423, -0.0173,  0.0100, -0.0776,  0.0241,  0.0384,
         -0.1208,  0.0751]], grad_fn=<AddmmBackward0>)
tensor(1)


In [16]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [17]:
learning_rate=0.001
training_epochs=30
batch_size=100

In [18]:
mnist_train=dsets.MNIST(root='data/MNIST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='data/MNIST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [19]:
print(mnist_train)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data/MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [20]:
print(mnist_test)

Dataset MNIST
    Number of datapoints: 10000
    Root location: data/MNIST_data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [21]:
data_loader=DataLoader(dataset=mnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=nn.Linear(7*7*64, 10, bias=True) # 완전연결층
        nn.init.xavier_uniform_(self.fc.weight) # 완전연결층 한정 가중치 초기화

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [23]:
model=CNN().to(device)
criterion=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
total_batch=len(data_loader)
print(total_batch)

600


In [25]:
for epoch in range(training_epochs):
    avg_cost=0
    avg_accuracy=0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost=criterion(y_hat, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print(epoch, avg_cost)

0 tensor(0.2256, grad_fn=<AddBackward0>)
1 tensor(0.0630, grad_fn=<AddBackward0>)
2 tensor(0.0463, grad_fn=<AddBackward0>)
3 tensor(0.0374, grad_fn=<AddBackward0>)
4 tensor(0.0313, grad_fn=<AddBackward0>)
5 tensor(0.0260, grad_fn=<AddBackward0>)
6 tensor(0.0217, grad_fn=<AddBackward0>)
7 tensor(0.0180, grad_fn=<AddBackward0>)
8 tensor(0.0162, grad_fn=<AddBackward0>)
9 tensor(0.0132, grad_fn=<AddBackward0>)
10 tensor(0.0097, grad_fn=<AddBackward0>)
11 tensor(0.0098, grad_fn=<AddBackward0>)
12 tensor(0.0086, grad_fn=<AddBackward0>)
13 tensor(0.0070, grad_fn=<AddBackward0>)
14 tensor(0.0071, grad_fn=<AddBackward0>)
15 tensor(0.0054, grad_fn=<AddBackward0>)
16 tensor(0.0041, grad_fn=<AddBackward0>)
17 tensor(0.0048, grad_fn=<AddBackward0>)
18 tensor(0.0050, grad_fn=<AddBackward0>)
19 tensor(0.0036, grad_fn=<AddBackward0>)
20 tensor(0.0029, grad_fn=<AddBackward0>)
21 tensor(0.0039, grad_fn=<AddBackward0>)
22 tensor(0.0043, grad_fn=<AddBackward0>)
23 tensor(0.0031, grad_fn=<AddBackward0>)
24